In [1]:
from todoist_habit_tracker.todoist_connection import TodoistConnection
from todoist_habit_tracker.habit import Habit
import os
import pandas as pd
from datetime import datetime

def init_todoist():
    dirname = os.path.abspath(os.path.abspath(''))
    filename = os.path.join(os.path.split(dirname)[0], '.config/.todoist.txt')
    todoist = TodoistConnection.from_config_file(filename)
    return todoist

def append_data(succes_dict):
    dirname = os.path.abspath(os.path.abspath(''))
    filename = os.path.join(os.path.split(dirname)[0], '.log/daily_tasks.csv')
    try:
        df = pd.read_csv(filename)
    except FileNotFoundError:
        return pd.DataFrame()

In [2]:
LABEL = 'habit_automatic'
todoist = init_todoist()

TEST_PROJECT='habit_tracker_test'
project_id = todoist.get_project_by_name(TEST_PROJECT)['id']
# open data file
label_id = todoist.get_label_by_name(LABEL)['id']
habits = todoist.filter_tasks(lambda task: (label_id in task['labels']) and (project_id != task['project_id']))
habit_objects = [Habit(habit, todoist, label_id, None) for habit in habits]
succes = {}
for habit in habit_objects:
    succes[str(habit)] =  habit.done()

print(succes)



{'registration mode': False, 'mail inbox check': True, 'daily review': True, 'stretching 1': True, 'apply ointment evening': False, 'apply ointment morning': True, 'meditate': True, 'update crm': True, 'stretching 2': False, 'stretchen 3': False, 'empty desk inbox daily': True, 'shave': True, 'eat fruit 1': True, 'eat fruit 2': True, 'check news': True, 'home workout': False, 'breakfast outside of bed': True}


In [3]:
def append_data(succes_dict):
    dirname = os.path.abspath(os.path.abspath(''))
    filename = os.path.join(os.path.split(dirname)[0], 'log/daily_tasks.csv')
    try:
        df = pd.read_csv(filename, index_col=0)
        df = df.append(pd.DataFrame(succes, index=[datetime.today().strftime('%d/%m/%Y')]))
    except FileNotFoundError:
        df = pd.DataFrame(succes, index=[datetime.today().strftime('%d/%m/%Y')])
        df.index.name = 'date'
    df.to_csv(filename)



In [4]:
year=2021
month=1
comp_task = [todoist.get_completed_tasks(year=year, month=month, day=day)['items'] for day in range(1, 32)]
comp_task.append(todoist.get_completed_tasks(year=year, month=2, day=1)['items'])
comp_task.append(todoist.get_completed_tasks(year=year, month=2, day=2)['items'])

In [8]:
comp_task[-3]

[{'completed_date': '2021-01-31T21:56:57Z',
  'content': 'stretching 2 @habit',
  'id': 4239660792,
  'meta_data': None,
  'project_id': 2245640739,
  'task_id': 4538571446,
  'user_id': 15358147},
 {'completed_date': '2021-01-31T21:56:56Z',
  'content': 'apply ointment evening @habit @evening',
  'id': 4239660784,
  'meta_data': None,
  'project_id': 2245640739,
  'task_id': 4538571428,
  'user_id': 15358147},
 {'completed_date': '2021-01-31T21:13:16Z',
  'content': 'daily review @habit @evening @computer',
  'id': 4239579415,
  'meta_data': None,
  'project_id': 2245640837,
  'task_id': 4538571409,
  'user_id': 15358147},
 {'completed_date': '2021-01-31T21:13:13Z',
  'content': 'update exist',
  'id': 4239579311,
  'meta_data': None,
  'project_id': 2245640837,
  'task_id': 4538571419,
  'user_id': 15358147},
 {'completed_date': '2021-01-31T21:10:25Z',
  'content': 'read week focus',
  'id': 4239573797,
  'meta_data': None,
  'project_id': 2245640837,
  'task_id': 4538571415,
  'user

In [9]:
import re
days = {1:31, 2:2}
results = {}
for habit in habit_objects:
    results[str(habit)]= {}
    for month in days:
        for day in range(1, days[month]+1):
            contents = [re.sub('@.* ', '', c['content']) for c in comp_task[(month-1)*31 + day-1]]
            contents = [re.sub('@.*', '', c).strip() for c in contents]
            results[str(habit)][datetime(year=2021, month=month, day=day).strftime('%d/%m/%Y')] = habit.content in contents

In [35]:
results

{'registration mode': {'01/01/2021': True,
  '02/01/2021': False,
  '03/01/2021': True,
  '01/02/2021': True,
  '02/02/2021': False},
 'mail inbox check': {'01/01/2021': True,
  '02/01/2021': True,
  '03/01/2021': True,
  '01/02/2021': True,
  '02/02/2021': True},
 'daily review': {'01/01/2021': False,
  '02/01/2021': False,
  '03/01/2021': False,
  '01/02/2021': False,
  '02/02/2021': False},
 'stretching 1': {'01/01/2021': True,
  '02/01/2021': True,
  '03/01/2021': True,
  '01/02/2021': True,
  '02/02/2021': True},
 'apply ointment evening': {'01/01/2021': True,
  '02/01/2021': False,
  '03/01/2021': True,
  '01/02/2021': True,
  '02/02/2021': False},
 'apply ointment morning': {'01/01/2021': True,
  '02/01/2021': True,
  '03/01/2021': True,
  '01/02/2021': True,
  '02/02/2021': True},
 'meditate': {'01/01/2021': True,
  '02/01/2021': False,
  '03/01/2021': True,
  '01/02/2021': True,
  '02/02/2021': False},
 'update crm': {'01/01/2021': True,
  '02/01/2021': False,
  '03/01/2021': 

In [10]:
res = pd.DataFrame(results)
res

,registration mode,mail inbox check,daily review,stretching 1,apply ointment evening,apply ointment morning,meditate,update crm,stretching 2,stretchen 3,empty desk inbox daily,shave,eat fruit 1,eat fruit 2,check news,home workout,breakfast outside of bed
01/01/2021,True,True,False,True,True,True,True,True,True,True,True,True,False,False,False,False,False
02/01/2021,False,True,False,True,False,True,False,False,True,False,True,False,False,False,False,False,False
03/01/2021,True,True,False,True,True,True,True,True,True,True,True,True,False,False,False,False,False
04/01/2021,False,True,False,True,True,True,True,True,True,True,True,True,False,False,False,False,False
05/01/2021,False,True,False,True,True,True,True,True,True,True,True,True,False,False,False,False,False
06/01/2021,False,True,False,True,True,True,True,True,True,False,True,True,False,False,False,False,False
07/01/2021,True,True,False,True,True,True,True,True,True,True,True,True,False,False,False,False,False
08/01/2021,True,True,False,True,True,True,True,True,True,False,True,True,False,False,False,False,False
09/01/2021,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False
10/01/2021,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False


In [ ]:
res.rolling(window=7, min_periods=7).mean()

In [11]:
dirname = os.path.abspath(os.path.abspath(''))
filename = os.path.join(os.path.split(dirname)[0], 'log/daily_tasks.csv')
res.to_csv(filename)


In [ ]:
df = pd.read_csv(filename, index_col=0)
# new_columns = df.columns.values
# new_columns[0] = 'Date'

In [14]:
def df_to_drive(folder, drive_api, df, title):
    folder_id = drive_api.ListFile({'q': "title='" + folder + "' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()[0]['id']
    f = drive_api.CreateFile({'title': title, 'parents': [{'id': folder_id}]})
    dirname = os.path.abspath(os.path.abspath(''))
    filename = os.path.join(os.path.split(dirname)[0], 'log/{0}'.format(title))
    df.to_csv(filename)
    f.SetContentFile(filename)
    f.Upload({'convert': False})

In [16]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()
dirname = os.path.abspath(os.path.abspath(''))
filename = os.path.join(os.path.split(dirname)[0], '.config/.drive_credentials.json')
print(filename)
gauth.LoadCredentialsFile(filename)
if gauth.access_token_expired:
    gauth.Refresh()
drive = GoogleDrive(gauth)
df_to_drive('QS', drive, res.astype(int), 'daily_tasks.csv')




/home/rutger/PycharmProjects/todoist_habit_tracker/.config/.drive_credentials.json
